In [5]:
import requests
import sys
import json 
import csv
# import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
# databse 
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import insert
import pymysql

pymysql.install_as_MySQLdb()
engine = create_engine('mysql://root:@localhost/web_data')
conn = engine.connect()
# metadata = db.MetaData()
meta = MetaData()
products = db.Table('osudhpotro', meta, autoload=True, autoload_with=engine)
category = db.Table('osudhpotro_category', meta, autoload=True, autoload_with=engine)
##### for product
s = products.select()
myresult = conn.execute(s)
##### for category
# c = category.select()
# mycategory = conn.execute(c)
#database conn end
# for x in myresult:
#     print(x[2])
#     quit()
sys.path.insert(0, 'usr/lib/chromium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://osudpotro.com/category/buy-over-the-counter-medicine-online-in-dhaka") #otc
r = driver.page_source
soup = BeautifulSoup(r, 'html.parser')
url_prefix = "https://osudpotro.com"
cat_link =[]
cat_name = []
all_data = dict();
for all_cat_div in soup.find_all('div', {'class':'all-prod-list-cont'}):
    for one_cat in all_cat_div.find_all('div', {'class':'single-prod-cont product-item'}):
        for pro_info in one_cat.find_all('div', {'class':'product-info-wrap'}):
            for cat_all_link in pro_info.find_all('a', {'class':'img-placeholder nav-link'}):
                cat_link.append(url_prefix+cat_all_link['href'])
                
        for name in one_cat.find_all('label', {'class':'productTitle'}):
            cat_name.append(name.text)
#             ins = category.insert().values(category_name = cat_name , 
#                                             category_link = cat_link)
#             reslt = conn.execute(ins)
# print(cat_name)
# print(cat_link)
for ct_link in cat_link:
#     print('hello')
    c = category.select()
    mycategory = conn.execute(c)
    driver.get(ct_link)
    r = driver.page_source
    soup = BeautifulSoup(r, 'html.parser')
    pro_link = []
    datas = [];
    all_data['category_link'] = ct_link
#     url_prefix = "https://osudpotro.com"
    for dv in soup.find_all('div', {'class':'disease-inner-page container'}):
        for name_row in dv.find_all('div', {'class':'disease-details row'}):        
            for nm in name_row.find_all('div', {'class':'disease-name-detail col-lg-8'}):       
                for ct_nm in nm.find_all('h3'):
                    category_name = ct_nm.text
        for rw in dv.find_all('div', {'class': 'row'}):
            for cl in rw.find_all('div', {'class':'col'}):
                for all_prod in cl.find_all('div', {'class':'all-prod-list-cont'}):
                    for single_prod in all_prod.find_all('div', {'class':'single-prod-cont product-item'}):
                        for pro_info in single_prod.find_all('div', {'class':'product-info-wrap'}):
                            for pro_a in pro_info.find_all('a',{'class':'img-placeholder nav-link'}):
                                pro_link.append(url_prefix + pro_a['href'])
    all_data['category_name'] = category_name
    cat_not_match = 0
    for ct in mycategory:
#         print('cat_in_db:', ct[2])
#         print('cat:',all_data['category_link'])
        if ct[2] ==  all_data['category_link']:
            cat_not_match = 0
            break;
        else:
             cat_not_match += 1
    if cat_not_match != 0:
#         print(all_data['category_link'])
        cat_ins = category.insert().values(category_name = all_data['category_name'], 
                                                category_link = all_data['category_link'])
        reslt = conn.execute(cat_ins)
#     print(pro_link)
    if pro_link == []:
        status = 0 #no data found
#         print('status', status)
#         print(ct_link);
#         for ct_l in myresult:
#             if ct_l[2] == ct_link:
#                 print('cat match')
#                 break
#             else:
#                 ins = products.insert().values(category_name = all_data['category_name'] , 
#                                             category_link = all_data['category_link'], status=0 )
#                 reslt = conn.execute(ins)
#         for ct_link in myresult:
#             print(ct_link[2])
#             if ct_link[2] == all_data['category_link']:
#                 continue;
#             else:
#                 ins = products.insert().values(category_name = all_data['category_name'] , 
#                                                            category_link = all_data['category_link'], status=0 )
#                 reslt = conn.execute(ins)
    # print(pro_link)
    else:        
        for u in pro_link:
            status = 1
    #         driver.get(pr)
    #         r = driver.page_source
    #         soup = BeautifulSoup(r, 'html.parser')
    #         for u in url: 
            print('url', u)
            product_images =[];
            discount_percent = [];
            slider_images = [];
            title_name = [];
            med_type = [];
            a_reference = [];
            a_text = [];
            generic_name = [];
            special_price_name = [];
            special_price_val = [];
            regular_price = [];
            option_name = [];
            product_description = [];
            recommended_products = [];
            disclaimer = [];
            reference_first_part = "https://osudpotro.com/";
            recom_img_link = [];       
            all_data['product_url'] = []
            all_data['product_images'] = []
            all_data['img_discount_percent']= []
            all_data['slider_images']= []
            all_data['medicine type']= []
            all_data['medicine name']= []
            all_data['generic_link']  = []
            all_data['generic_name'] = []
            all_data['manufacturer name'] = []
            all_data['manufacturer link'] = []
            all_data['special_price'] = []
            all_data['regular_price'] = []
            all_data['variation_option_name'] = []
            all_data['buy_btn_text']= []
            all_data['cart_btn_txt']= []
            all_data['product_description']= []
            all_data['recommended_products']= []
            all_data['disclaimer']= []
            driver.get(u)
            r = driver.page_source
            soup = BeautifulSoup(r, 'html.parser')
            all_data['product_url'] = u
            for img_div in soup.find_all('div',{'class': 'product-image-cont-compo'}):
            ######################### product_images ###################
                for im in img_div.find_all('img'):
                    product_images.append(im['src'])
                all_data['product_images'] = product_images
            ########################## discount_percent ####################
                for dis in img_div.find_all('div',{'class':'discount-percent'}):
                    discount_percent.append(dis.text)
                all_data['img_discount_percent'] = discount_percent
            ######################### slider_images ###################
            for sld in soup.find_all('div', {'class': 'image-swipper-cont'}):
                for sld_img_div in sld.find_all('div', {'class': 'swiper-slide image-swipper swiper-slide-active'}):
                    for sld_im in sld_img_div.find_all('img'):
                        slider_images.append(sld_im['src'])
                    all_data['slider_images'] = slider_images
            for maindiv in soup.find_all('div', {'class': 'product-info-cont-compo product-details-details-inner-page col-lg-8'}):
                    for title_div in maindiv.find_all('div', {'class':'row'}):
                        for title_col in title_div.find_all('div', {'class':'col-lg-9'}):
                        ######################### [h1] medicine name & type ###################
                            for title in title_col.find_all('h1', { 'class' : 'product-info-heading-compo'}):                    
                                for title_span in title.find_all('span', {'class' : 'sku'}):
                                    med_type.append(title_span.text)
                                    all_data['medicine type'] = med_type
                                title.find('span').extract()
                                title_name.append(title.getText().strip())
                            all_data['medicine name'] = title_name
                        ######################### generic name ###################      
                            for generics in title_col.find_all('div', {'class':'generes-wrap'}):
                                for gn_link in generics.find_all('a', {'class':'nav-link'}):
                                    generic_link = gn_link['href']
                                    for gn_name in gn_link.find_all('span'):
                                        generic_name.append(gn_name.text)
                            all_data['generic_link'] = generic_link
                            all_data['generic_name'] = generic_name;
                        ######################### [p] manufacturer name, link ###################
                            for p_manufacturer in title_col.find_all('p', {'class':'menufacturer'}):
                                for manf_link in p_manufacturer.find_all('a', {'class' : 'nav-link'}):
                                    a_reference.append(reference_first_part+manf_link['href']);
                                    a_text.append(manf_link.text)
                                all_data['manufacturer name'] = a_text
                                all_data['manufacturer link'] = a_reference                        

            data_price = json.loads(soup.find('script', id="__NEXT_DATA__").text)
            inv = data_price['props']['initialProps']['pageProps']['productData']['inventory']
            for prices in inv:
                special_price_val.append(prices["price"])
                regular_price.append(prices["regular_price"])
                option_name.append(prices['variation_option_name'])
            all_data['special_price'] = special_price_val
            all_data['regular_price'] = regular_price
            all_data['variation_option_name'] = option_name
        ######################### buttons ###################
            for buttons in soup.find_all('div',{'class':'prod-control-buttons d-flex justify-content-between'}):
                for buy_button in buttons.find_all('button', {'class':'btn buy-more-button ml-3 mr-2'}):
                    buy_button.find('i').extract()
                    buy_btn_text = buy_button.text
                all_data['buy_btn_text'] = buy_btn_text
                for cart_butn in buttons.find_all('button', { 'class' : 'btn btn-secondary details-checkout-btn ml-2'}):
                    cart_butn.find('i').extract()
                    cart_btn_txt = cart_butn.text            
                all_data['cart_btn_txt'] = cart_btn_txt
            for pro_des in soup.find_all('div', { 'class' : 'd-none d-sm-block row'}):
                product_description.append(pro_des)
            all_data['product_description'] = product_description

            for swp in soup.find_all('div', {'class':'jss3'}):
                for swp_wrap in swp.find_all('div', {'class':'swiper-container jss5 swiper-container-initialized swiper-container-horizontal swiper-container-pointer-events'}):
                    for sld in swp.find_all('div', {'class':'swiper-wrapper'}):
                        all_data['recommended_products'] = sld
            for disc_cont in soup.find_all('div' , { 'class': 'container'}):
                for disc_row in disc_cont.find_all('div', {'class':'row'}):
                    for disc_col in disc_row.find_all('div', {'class':'col'}):
                        for disc_p in disc_col.find_all('p', {'class':'product-disclaimer'}):
                            disclaimer.append(disc_p.text)   
                        all_data['disclaimer'] = disclaimer
            datas.append(all_data)
#             print(datas)
#             print(myresult)
            for x in myresult:
                myresult = conn.execute(s)
                match = 0 
#                 print('db data: ', x[3])
#                 print('url', all_data['product_url'])
#                 print('ca:',all_data['category_link'])
#                 exit()
                if x[3] == all_data['product_url']:
#                     al =1;
                    match = 0 
#                     print('product matched');
                    break
#                     continue;
                else:
                    match += 1
#                     print('not matched')
            if match != 0:
                print('insert')
                ins = products.insert().values(category_name = str(all_data['category_name']) , 
                                                   category_link = str(all_data['category_link']),
                                                   product_url =str(all_data['product_url']), 
                                                   product_images=str(all_data['product_images']),  
                                                   img_discount_percent=str(all_data['img_discount_percent']),
                                                   slider_images=str(all_data['slider_images']),
                                                   medicine_type=str(all_data['medicine type']),
                                                   medicine_name=str(all_data['medicine name']),
                                                   generic_link=str(all_data['generic_link']),
                                                   generic_name=str(all_data['generic_name']), 
                                                   manufacturer_name =str(all_data['manufacturer name']),
                                                   manufacturer_link =str(all_data['manufacturer link']), 
                                                   special_price=str(all_data['special_price']),
                                                   regular_price=str(all_data['regular_price']),
                                                   variation_option_name=str(all_data['variation_option_name']),
                                                   buy_btn_text=str(all_data['buy_btn_text']), 
                                                   product_description = str(all_data['product_description']),
                                                   recommended_products = str(all_data['recommended_products']), 
                                                   disclaimer= str(all_data['disclaimer']), status=1 
                                                    )
                reslt = conn.execute(ins)    
#                     break
#                     print('result', reslt)
#                     ins = products.insert().values(category_name = str(all_data['category_name']) , 
#                                                    category_link = str(all_data['category_link']),
#                                                    product_url =str(all_data['product_url']), 
#                                                    product_images=str(all_data['product_images']),                                                      
#                                                        img_discount_percent=str(all_data['img_discount_percent']),
#                                                        slider_images=str(all_data['slider_images']),
#                                                        medicine_type=str(all_data['medicine type']), 
#                                                        medicine_name=str(all_data['medicine name']),
#                                                        generic_link=str(all_data['generic_link']),
#                                                        generic_name=str(all_data['generic_name']), 
#                                                        manufacturer_name =str(all_data['manufacturer name']),
#                                                        manufacturer_link =str(all_data['manufacturer link']), 
#                                                        special_price=str(all_data['special_price']),regular_price=str(all_data['regular_price']), 
#                                                        variation_option_name=all_data['variation_option_name'], 
#                                                        buy_btn_text=all_data['buy_btn_text'], product_description = all_data['product_description'],
#                                                        recommended_products = all_data['recommended_products'], disclaimer=all_data['disclaimer'], status=1 )
# #                     print(ins)
#                     reslt = conn.execute(ins)
#     print(datas)
#     if status == 0:
# #         print('2')
#         for ct_l in myresult:
#             if ct_l[2] == ct_link:
#                 print('cat match')
# #             print(ct_l[2])

#         ins = products.insert().values(category_name = all_data['category_name'] , 
#                                                    category_link = all_data['category_link'], status=0 )
#         reslt = conn.execute(ins)
driver.close()
driver.quit()

url https://osudpotro.com/neocort-ointment
url https://osudpotro.com/betson-cream
url https://osudpotro.com/betson-ointment
url https://osudpotro.com/betameson-cream
url https://osudpotro.com/prosalic-lotion-25-ml-scalp-lotion
url https://osudpotro.com/betamesal-ointment
url https://osudpotro.com/calamin-m-100-ml-lotion
url https://osudpotro.com/nebanol-20-ointment
url https://osudpotro.com/cevion-1000-tablet
url https://osudpotro.com/bet-a-050-tablet
url https://osudpotro.com/nosomist-15-ml-nasal-drop
url https://osudpotro.com/nasal-drop
url https://osudpotro.com/afrin-005-10-ml-nasal-drop
url https://osudpotro.com/p-d-nasal-drop
url https://osudpotro.com/rynex-005-10-ml-nasal-drop
url https://osudpotro.com/antazol-005-nasal-drop
url https://osudpotro.com/solo-10-ml-nasal-drop
url https://osudpotro.com/norsol-25-ml-lotion
url https://osudpotro.com/xylovin-05-10-ml-nasal-drop
url https://osudpotro.com/xylovin-01-nasal-drop
url https://osudpotro.com/xylomet-01-nasal-drop
url https://osu

url https://osudpotro.com/nispore-150-capsule
url https://osudpotro.com/nispore-200-capsule
url https://osudpotro.com/nispore-50-capsule
url https://osudpotro.com/nispore-ophthalmic-solution
url https://osudpotro.com/nispore-35-ml-powder-for-suspension
url https://osudpotro.com/oroconazol-oral-gel
url https://osudpotro.com/topicazole-10-cream
url https://osudpotro.com/topicazole-plus-cream
url https://osudpotro.com/diflu-35-ml-powder-for-suspension
url https://osudpotro.com/diflu-150-capsule
url https://osudpotro.com/diflu-200-capsule
url https://osudpotro.com/diflu-50-capsule
url https://osudpotro.com/fusidate-h-10-cream
url https://osudpotro.com/mycon-15-oral-gel
url https://osudpotro.com/candex-30-ml-oral-suspension
url https://osudpotro.com/oroconazol-oral-gel
url https://osudpotro.com/topicazole-10-cream
url https://osudpotro.com/topicazole-plus-cream
url https://osudpotro.com/mycon-15-oral-gel
url https://osudpotro.com/mic-hc-10-cream
url https://osudpotro.com/candoral-15-gel
url

In [ ]:
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import insert
import pymysql

pymysql.install_as_MySQLdb()
engine = create_engine('mysql://root:@localhost/web_data')
conn = engine.connect()
# metadata = db.MetaData()
meta = MetaData()
products = db.Table('osudhpotro', meta, autoload=True, autoload_with=engine)
s = products.select()
myresult = conn.execute(s)
for x in myresult:
    print(x[2])
#                 exit()
    if x[2] == all_data['product_url']:
        print('products matched', all_data['product_url']);
#                     continue;
    else:
        print('insert')

In [8]:
import requests
import sys
import json 
import csv
# import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
# databse 
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import insert
import pymysql

pymysql.install_as_MySQLdb()
engine = create_engine('mysql://root:@localhost/web_data')
conn = engine.connect()
# metadata = db.MetaData()
meta = MetaData()
products = db.Table('osudhpotro', meta, autoload=True, autoload_with=engine)
s = products.select()
myresult = conn.execute(s)
#database conn end
# for x in myresult:
#     print(x[2])
#     quit()
sys.path.insert(0, 'usr/lib/chromium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://osudpotro.com/category/buy-over-the-counter-medicine-online-in-dhaka")
r = driver.page_source
soup = BeautifulSoup(r, 'html.parser')
url_prefix = "https://osudpotro.com"
cat_link =[]
cat_name = []
all_data = dict();
for all_cat_div in soup.find_all('div', {'class':'all-prod-list-cont'}):
    for one_cat in all_cat_div.find_all('div', {'class':'single-prod-cont product-item'}):
        for pro_info in one_cat.find_all('div', {'class':'product-info-wrap'}):
            for cat_all_link in pro_info.find_all('a', {'class':'img-placeholder nav-link'}):
                cat_link.append(url_prefix+cat_all_link['href'])
        for name in one_cat.find_all('label', {'class':'productTitle'}):
            cat_name.append(name.text)
# print(cat_name)
# print(cat_link)
for ct_link in cat_link:
    
    driver.get(ct_link)
    r = driver.page_source
    soup = BeautifulSoup(r, 'html.parser')
    pro_link = []
    datas = [];
    all_data['category_link'] = ct_link
#     url_prefix = "https://osudpotro.com"
    for dv in soup.find_all('div', {'class':'disease-inner-page container'}):
        for name_row in dv.find_all('div', {'class':'disease-details row'}):        
            for nm in name_row.find_all('div', {'class':'disease-name-detail col-lg-8'}):       
                for ct_nm in nm.find_all('h3'):
                    category_name = ct_nm.text
        for rw in dv.find_all('div', {'class': 'row'}):
            for cl in rw.find_all('div', {'class':'col'}):
                for all_prod in cl.find_all('div', {'class':'all-prod-list-cont'}):
                    for single_prod in all_prod.find_all('div', {'class':'single-prod-cont product-item'}):
                        for pro_info in single_prod.find_all('div', {'class':'product-info-wrap'}):
                            for pro_a in pro_info.find_all('a',{'class':'img-placeholder nav-link'}):
                                pro_link.append(url_prefix + pro_a['href'])
    all_data['category_name'] = category_name
    if pro_link == []:
        status = 0 #no data found
#         print('status', status)
#         ins = products.insert().values(category_name = all_data['category_name'] , 
#                                                    category_link = all_data['category_link'], status=0 )
#         reslt = conn.execute(ins)
    # print(pro_link)
    else:
        
        for u in pro_link:
            status = 1
    #         driver.get(pr)
    #         r = driver.page_source
    #         soup = BeautifulSoup(r, 'html.parser')
    #         for u in url: 
#             print('url', u)
            product_images =[];
            discount_percent = [];
            slider_images = [];
            title_name = [];
            med_type = [];
            a_reference = [];
            a_text = [];
            generic_name = [];
            special_price_name = [];
            special_price_val = [];
            regular_price = [];
            option_name = [];
            product_description = [];
            recommended_products = [];
            disclaimer = [];
            reference_first_part = "https://osudpotro.com/";
            recom_img_link = [];       
            all_data['product_url'] = []
            all_data['product_images'] = []
            all_data['img_discount_percent']= []
            all_data['slider_images']= []
            all_data['medicine type']= []
            all_data['medicine name']= []
            all_data['generic_link']  = []
            all_data['generic_name'] = []
            all_data['manufacturer name'] = []
            all_data['manufacturer link'] = []
            all_data['special_price'] = []
            all_data['regular_price'] = []
            all_data['variation_option_name'] = []
            all_data['buy_btn_text']= []
            all_data['cart_btn_txt']= []
            all_data['product_description']= []
            all_data['recommended_products']= []
            all_data['disclaimer']= []
            driver.get(u)
            r = driver.page_source
            soup = BeautifulSoup(r, 'html.parser')
            all_data['product_url'] = u
            for img_div in soup.find_all('div',{'class': 'product-image-cont-compo'}):
            ######################### product_images ###################
                for im in img_div.find_all('img'):
                    product_images.append(im['src'])
                all_data['product_images'] = product_images
            ########################## discount_percent ####################
                for dis in img_div.find_all('div',{'class':'discount-percent'}):
                    discount_percent.append(dis.text)
                all_data['img_discount_percent'] = discount_percent
            ######################### slider_images ###################
            for sld in soup.find_all('div', {'class': 'image-swipper-cont'}):
                for sld_img_div in sld.find_all('div', {'class': 'swiper-slide image-swipper swiper-slide-active'}):
                    for sld_im in sld_img_div.find_all('img'):
                        slider_images.append(sld_im['src'])
                    all_data['slider_images'] = slider_images
            for maindiv in soup.find_all('div', {'class': 'product-info-cont-compo product-details-details-inner-page col-lg-8'}):
                    for title_div in maindiv.find_all('div', {'class':'row'}):
                        for title_col in title_div.find_all('div', {'class':'col-lg-9'}):
                        ######################### [h1] medicine name & type ###################
                            for title in title_col.find_all('h1', { 'class' : 'product-info-heading-compo'}):                    
                                for title_span in title.find_all('span', {'class' : 'sku'}):
                                    med_type.append(title_span.text)
                                    all_data['medicine type'] = med_type
                                title.find('span').extract()
                                title_name.append(title.getText().strip())
                            all_data['medicine name'] = title_name
                        ######################### generic name ###################      
                            for generics in title_col.find_all('div', {'class':'generes-wrap'}):
                                for gn_link in generics.find_all('a', {'class':'nav-link'}):
                                    generic_link = gn_link['href']
                                    for gn_name in gn_link.find_all('span'):
                                        generic_name.append(gn_name.text)
                            all_data['generic_link'] = generic_link
                            all_data['generic_name'] = generic_name;
                        ######################### [p] manufacturer name, link ###################
                            for p_manufacturer in title_col.find_all('p', {'class':'menufacturer'}):
                                for manf_link in p_manufacturer.find_all('a', {'class' : 'nav-link'}):
                                    a_reference.append(reference_first_part+manf_link['href']);
                                    a_text.append(manf_link.text)
                                all_data['manufacturer name'] = a_text
                                all_data['manufacturer link'] = a_reference                        

            data_price = json.loads(soup.find('script', id="__NEXT_DATA__").text)
            inv = data_price['props']['initialProps']['pageProps']['productData']['inventory']
            for prices in inv:
                special_price_val.append(prices["price"])
                regular_price.append(prices["regular_price"])
                option_name.append(prices['variation_option_name'])
            all_data['special_price'] = special_price_val
            all_data['regular_price'] = regular_price
            all_data['variation_option_name'] = option_name
        ######################### buttons ###################
            for buttons in soup.find_all('div',{'class':'prod-control-buttons d-flex justify-content-between'}):
                for buy_button in buttons.find_all('button', {'class':'btn buy-more-button ml-3 mr-2'}):
                    buy_button.find('i').extract()
                    buy_btn_text = buy_button.text
                all_data['buy_btn_text'] = buy_btn_text
                for cart_butn in buttons.find_all('button', { 'class' : 'btn btn-secondary details-checkout-btn ml-2'}):
                    cart_butn.find('i').extract()
                    cart_btn_txt = cart_butn.text            
                all_data['cart_btn_txt'] = cart_btn_txt
            for pro_des in soup.find_all('div', { 'class' : 'd-none d-sm-block row'}):
                product_description.append(pro_des)
            all_data['product_description'] = product_description

            for swp in soup.find_all('div', {'class':'jss3'}):
                for swp_wrap in swp.find_all('div', {'class':'swiper-container jss5 swiper-container-initialized swiper-container-horizontal swiper-container-pointer-events'}):
                    for sld in swp.find_all('div', {'class':'swiper-wrapper'}):
                        all_data['recommended_products'] = sld
            for disc_cont in soup.find_all('div' , { 'class': 'container'}):
                for disc_row in disc_cont.find_all('div', {'class':'row'}):
                    for disc_col in disc_row.find_all('div', {'class':'col'}):
                        for disc_p in disc_col.find_all('p', {'class':'product-disclaimer'}):
                            disclaimer.append(disc_p.text)   
                        all_data['disclaimer'] = disclaimer
            datas.append(all_data)
#             print(datas)
#             print(myresult)
#             print('insert')
            ins = products.insert().values(category_name = str(all_data['category_name']) , 
                                                   category_link = str(all_data['category_link']),
                                                   product_url =str(all_data['product_url']), 
                                                   product_images=str(all_data['product_images']),  
                                                   img_discount_percent=str(all_data['img_discount_percent']),
                                                   slider_images=str(all_data['slider_images']),
                                                   medicine_type=str(all_data['medicine type']),
                                                   medicine_name=str(all_data['medicine name']),
                                                   generic_link=str(all_data['generic_link']),
                                                   generic_name=str(all_data['generic_name']), 
                                                   manufacturer_name =str(all_data['manufacturer name']),
                                                   manufacturer_link =str(all_data['manufacturer link']), 
                                                   special_price=str(all_data['special_price']),
                                                   regular_price=str(all_data['regular_price']),
                                                   variation_option_name=str(all_data['variation_option_name']),
                                                   buy_btn_text=str(all_data['buy_btn_text']), 
                                                   product_description = str(all_data['product_description']),
                                                   recommended_products = str(all_data['recommended_products']), 
                                                   disclaimer= str(all_data['disclaimer']), status=1 
                                                    )
            reslt = conn.execute(ins)    
#             print('result', reslt)
            
                    
#                     ins = products.insert().values(category_name = str(all_data['category_name']) , 
#                                                    category_link = str(all_data['category_link']),
#                                                    product_url =str(all_data['product_url']), 
#                                                    product_images=str(all_data['product_images']),                                                      
#                                                        img_discount_percent=str(all_data['img_discount_percent']),
#                                                        slider_images=str(all_data['slider_images']),
#                                                        medicine_type=str(all_data['medicine type']), 
#                                                        medicine_name=str(all_data['medicine name']),
#                                                        generic_link=str(all_data['generic_link']),
#                                                        generic_name=str(all_data['generic_name']), 
#                                                        manufacturer_name =str(all_data['manufacturer name']),
#                                                        manufacturer_link =str(all_data['manufacturer link']), 
#                                                        special_price=str(all_data['special_price']),regular_price=str(all_data['regular_price']), 
#                                                        variation_option_name=all_data['variation_option_name'], 
#                                                        buy_btn_text=all_data['buy_btn_text'], product_description = all_data['product_description'],
#                                                        recommended_products = all_data['recommended_products'], disclaimer=all_data['disclaimer'], status=1 )
#                     print(ins)
#                     reslt = conn.execute(ins)
#     print(datas)
#     if status == 0:
#         ins = products.insert().values(category_name = all_data['category_name'] , 
#                                                    category_link = all_data['category_link'], status=0 )
#         reslt = conn.execute(ins)
driver.close()
driver.quit()

status 0
url https://osudpotro.com/neocort-ointment
insert
result <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f330c927dc0>
url https://osudpotro.com/betson-cream
insert
result <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f330d1e3ac0>
url https://osudpotro.com/betson-ointment
insert
result <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f330c91fbb0>
url https://osudpotro.com/betameson-cream
insert
result <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f330c927e20>
url https://osudpotro.com/prosalic-lotion-25-ml-scalp-lotion
insert
result <sqlalchemy.engine.cursor.LegacyCursorResult object at 0x7f330c9fa040>
url https://osudpotro.com/betamesal-ointment


KeyboardInterrupt: 